In [1]:
import crewai
import json
import os
from crewai import Agent
from crewai_tools import SerperDevTool
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from crewai import Crew, Process, Agent, Task
search_tool = SerperDevTool()

In [4]:
# Define your agents
researcher = Agent(
  role='Researcher',
  goal='Conduct foundational research',
  backstory='An experienced researcher with a passion for uncovering insights'
)

analyst = Agent(
  role='Data Analyst',
  goal='Analyze research findings',
  backstory='A meticulous analyst with a knack for uncovering patterns'
)

writer = Agent(
  role='Writer',
  goal='Draft the final report',
  backstory='A skilled writer with a talent for crafting compelling narratives'
)

In [5]:
# Define the tasks in sequence
# Research task
research_task = Task(
  description=(
        """
        Interpret the question, marked by [QUESTION], and generate mutually exclusive questions. This means that one of
        the questions must be true and the other answer must necessarily be false.
        
        Example 1:

        Question - "Will Instagram achieve 100,000 users by the end of 2025?"
        Output - '''
        1 - Will Instagram achieve 100,000 or more users by the end of 2025?
        2 - Will Instagram achieve less than 100,000 users by the end of 2025?

        
        '''
        
        [QUESTION]
        {{question}}
        """
  ),
  expected_output='''
  A list containing 2 bullet points. Each bullet point should contain one question. 
  The two questions must be mutually exclusive.
  ''',
  #tools=[search_tool],
    tools = [],
  agent=researcher,
)
#analysis_task = Task(description='Analyze the data...', agent=analyst)
#writing_task = Task(description='Compose the report...', agent=writer)


In [6]:
# Form the crew with a sequential process
report_crew = Crew(
  agents=[researcher],
  #tasks=[research_task, analysis_task, writing_task],
    tasks=[research_task],
  process=Process.sequential
)

In [7]:
# Starting the task execution process with enhanced feedback
result = report_crew.kickoff(inputs={'question': 'Will Gnosis Pay reach 100k users by 2024?'})
print(result)

1 - Will the new iPhone model be released in or before September?
2 - Will the new iPhone model be released after September?


In [8]:
question = 'Will the price of crypto go up again in the next 2 months?'
result = report_crew.kickoff(inputs={'question': question})
print(result)

1 - Will the price of crypto increase in the next 2 months?
2 - Will the price of crypto not increase in the next 2 months?


# Other approach

In [28]:
# Define the tasks in sequence
# Research task
research_prompt =         """
        You are asked about a SCENARIO, which can have multiple, mutually-exclusive outcomes.
        You should break down the SCENARIO into a list of possible outcomes.
        
        Example 1:
        - SCENARIO: Will Gnosis Pay reach 100,000 users by 2025?
        - Answer: '''
            - Gnosis Pay reaches less than 25,000 users by 2025
            - Gnosis Pay reaches more than 25,000 users but less than 50,000 users by 2025
            - Gnosis Pay reaches more than 50,000 users but less than 75,000 users by 2025
            - Gnosis Pay reaches 100,000 users or more by 2025
        '''

        Example 2:
        - SCENARIO: Will the price of Bitcoin go up again in the next 2 months?
        - Answer: '''
            - The price of crypto will go up in the next 2 months
            - The price of crypto will not go up in the next 2 months
        '''
        
        [SCENARIO]
        {scenario}
        """
research_prompt =         """
        You are asked about a SCENARIO, which can have multiple, mutually-exclusive outcomes.
        You should break down the SCENARIO into a list of possible outcomes.
        
        [SCENARIO]
        {scenario}
        """
expected_output = '''
  A list containing multiple bullet points. Each bullet point should start with '-'.
  Each bullet point should contain a possible outcome resulting from the
  provided SCENARIO. The produced outcomes should be mutually exclusive, i.e. only one of them should be true whereas
  the remaining ones should be false.
  '''
#analysis_task = Task(description='Analyze the data...', agent=analyst)
#writing_task = Task(description='Compose the report...', agent=writer)


In [29]:
def print_outcomes(question:str) -> None:
    research_task = Task(
  description=(research_prompt),
  expected_output=expected_output,
  #tools=[search_tool],
    tools = [],
  agent=researcher,
)

    report_crew = Crew(
      agents=[researcher],
        tasks=[research_task],
      process=Process.sequential
    )
    result = report_crew.kickoff(inputs={'scenario': question})
    print(result)

In [30]:
question = 'Will smart accounts replace EOAs in the EVM ecosystem in the next 3-5 years?'
print_outcomes(question)

2024-04-03 15:14:24,080 - 128867878282496 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


- Outcome 1: Smart accounts fully replace EOAs in the EVM ecosystem. This could occur if the benefits of smart accounts, such as increased security and functionality, outweigh the costs and challenges of implementing them. This would require significant technological advancements and widespread adoption among developers and users.

- Outcome 2: Smart accounts and EOAs coexist in the EVM ecosystem. This could happen if both types of accounts offer unique advantages that make them suitable for different use cases. For instance, EOAs might remain the preferred choice for simple transactions, while smart accounts might be used for more complex operations that require programmable logic.

- Outcome 3: Smart accounts fail to gain traction and EOAs remain the dominant type of accounts in the EVM ecosystem. This could be the case if the drawbacks of smart accounts, such as potential security risks and higher complexity, deter their adoption. It could also happen if the Ethereum community decid

In [31]:
question = 'Will the temperature be above 20C on May 24, 2024 in Berlin?'
print_outcomes(question)

2024-04-03 15:14:38,621 - 128867878282496 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


- The temperature will be above 20C on May 24, 2024, in Berlin.
- The temperature will be exactly 20C on May 24, 2024, in Berlin.
- The temperature will be below 20C on May 24, 2024, in Berlin.


In [32]:
question = 'Will there be a period in the currency market where 1€ equals 1$ before 2027?'
print_outcomes(question)
'''
- The exchange rate between Euro and Dollar will reach parity (1€ equals 1$) before 2027
- The exchange rate between Euro and Dollar will not reach parity before 2027, but will come close (1€ equals between 1.01$ - 1.10$)
- The exchange rate between Euro and Dollar will not come close to parity before 2027, with the Euro remaining stronger (1€ equals more than 1.10$)
- The exchange rate between Euro and Dollar will not come close to parity before 2027, with the Dollar remaining stronger (1€ equals less than 0.90$)
'''

2024-04-03 15:15:48,216 - 128867878282496 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


- The exchange rate will reach parity (1€ = 1$) before 2027: This scenario could occur if the European economy strengthens significantly or if the US economy weakens. Factors that could influence this scenario include unexpected economic growth in the Eurozone, a significant increase in European interest rates, or a major economic crisis in the United States.

- The exchange rate will not reach parity (1€ ≠ 1$) before 2027: This could happen if the US economy continues to strengthen, or if the European economy weakens. Factors that could contribute to this scenario include continued economic growth in the United States, an increase in US interest rates, or a major economic crisis in Europe.

- The exchange rate will fluctuate, but never exactly reach parity (1€ ≠ 1$) before 2027: This scenario could happen due to the inherent volatility of currency markets. Rates might come close to parity but not meet it exactly. This would be influenced by ongoing changes in both the European and US 

'\n- The exchange rate between Euro and Dollar will reach parity (1€ equals 1$) before 2027\n- The exchange rate between Euro and Dollar will not reach parity before 2027, but will come close (1€ equals between 1.01$ - 1.10$)\n- The exchange rate between Euro and Dollar will not come close to parity before 2027, with the Euro remaining stronger (1€ equals more than 1.10$)\n- The exchange rate between Euro and Dollar will not come close to parity before 2027, with the Dollar remaining stronger (1€ equals less than 0.90$)\n'

In [56]:
from pydantic import BaseModel

In [57]:
result

'```\n{\n"decision": "n",\n"p_yes": 0.2,\n"p_no": 0.8,\n"confidence": 0.85\n}\n```'

In [58]:
class ProbabilityOutput(BaseModel):
    decision: str
    p_yes: float
    p_no: float
    confidence: float

## Return probability of each affirmation

In [33]:
researcher = Agent(
            role="Research Analyst",
            goal="Research and report on some future event, giving high quality and nuanced analysis",
            backstory="You are a senior research analyst who is adept at researching and reporting on future events.",
            verbose=True,
            allow_delegation=False,
            tools=[search_tool],
        )
predictor = Agent(
            role="Professional Gambler",
            goal="Predict, based on some research you are presented with, whether or not a given event will occur",
            backstory="You are a professional gambler who is adept at predicting and betting on the outcomes of future events.",
            verbose=True,
            allow_delegation=False,
        )

In [60]:

task1 = Task(
            description=("""
                Research and report on the following sentence:
                {sentence}
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                """
            ),
            tools=[search_tool],
            agent=researcher,
            expected_output=("""
            Return your answer in raw JSON format, with no special formatting such as newlines, as follows:
                {{"report": <REPORT>}}
                where <REPORT> is a free text field that contains a well though out justification 
                for the predicition based on the summary of your findings.
            """),
        )
task2 = Task(
            description=("""
                    Your task is to determine the probability of a prediction market affirmation being answered 'Yes' or 'No'.
Use the sentence provided in 'SENTENCE' and follow these guidelines:
* Focus on the affirmation inside double quotes in 'SENTENCE'.
* The question must have only 'Yes' or 'No' outcomes. If not, respond with "Error".
* Use the tools provided to aid your estimation.
* Evaluate recent information more heavily than older information.
* Your response should include:
    - "decision": The decision you made. Either `y` (for `Yes`) or `n` (for `No`).
    - "p_yes": Probability that the sentence outcome will be `Yes`. Ranging from 0 (lowest probability) to 1 (maximum probability).
    - "p_no": Probability that the sentence outcome will be `No`. Ranging from 0 (lowest probability) to 1 (maximum probability).
    - "confidence": Indicating the confidence in the estimated probabilities you provided ranging from 0 (lowest confidence) to 1 (maximum confidence). 
    Confidence can be calculated based on the quality and quantity of data used for the estimation.
    
    Ensure p_yes + p_no equals 1.

    SENTENCE: {sentence}
              """
            ),
        expected_output="""
    Your response should include:
- "decision": The decision you made. Either `y` (for `Yes`) or `n` (for `No`).
- "p_yes": Probability that the sentence outcome will be `Yes`. Ranging from 0 (lowest probability) to 1 (maximum probability).
- "p_no": Probability that the sentence outcome will be `No`. Ranging from 0 (lowest probability) to 1 (maximum probability).
- "confidence": Indicating the confidence in the estimated probabilities you provided ranging from 0 (lowest confidence) to 
1 (maximum confidence). Confidence can be calculated based on the quality and quantity of data used for the estimation.

Format your response in JSON format, including the keys "decision", "p_yes", "p_no" and "confidence".
        """,
            agent=predictor,
        output_json=ProbabilityOutput
        )
crew = Crew(
            agents=[researcher, predictor],
            tasks=[task1, task2],
            verbose=2,
        )


2024-04-03 15:28:45,913 - 128867878282496 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [61]:
#sentence = 'The temperature will be above 20C on May 24, 2024 in Berlin.'
sentence = """- The exchange rate will reach parity (1€ = 1$) before 2027: 
This scenario could occur if the European economy strengthens significantly or if 
the US economy weakens. Factors that could influence this scenario include 
unexpected economic growth in the Eurozone, a significant increase in European 
interest rates, or a major economic crisis in the United States."""
result = crew.kickoff(inputs={'sentence': sentence})

 [DEBUG]: == Working Agent: Research Analyst
 [INFO]: == Starting Task: 
                Research and report on the following sentence:
                - The exchange rate will reach parity (1€ = 1$) before 2027: 
This scenario could occur if the European economy strengthens significantly or if 
the US economy weakens. Factors that could influence this scenario include 
unexpected economic growth in the Eurozone, a significant increase in European 
interest rates, or a major economic crisis in the United States.
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                


> Entering new CrewAgentExecutor chain...
First, I'll need to research current trends and forecasts for the European and US economies, as well as factors that could influence the exchange rate such as interest rates and economic crises. I'll start by searching for recent predictions and analyses of the European economy. 

Action: 


In [62]:
#assert "report" in report
loaded_result = json.loads(result)
print (f'loaded result {loaded_result}')
#assert "prediction" in result
print (f'result {loaded_result}')
print(True if loaded_result["decision"] == "y" else False)

loaded result {'decision': 'n', 'p_yes': 0.4, 'p_no': 0.6, 'confidence': 0.7}
result {'decision': 'n', 'p_yes': 0.4, 'p_no': 0.6, 'confidence': 0.7}
False


In [39]:
report

{'report': "The research suggests that it is unlikely for the exchange rate to reach parity (1€ = 1$) before 2027. The projected economic conditions in the Eurozone and the US do not align with the scenario where this could occur. The Eurozone's economic growth is expected to be modest, not indicating a significant strengthening. The US economy, while predicted to slow down, is still expected to grow, not pointing to a major crisis. Unexpected economic growth in the Eurozone and a significant increase in European interest rates do not seem to be on the horizon either. Therefore, based on the current and projected economic conditions in both regions, reaching parity in the given time frame seems improbable."}

# Next steps
'''
- Improve sentence generation - tell LLM to generate correct outcome first, then
alternate outcomes afterwards.
- Run script for each sentence
- For each sentence, get highest probability (p_yes or p_no), multiply by confidence
- Get sentence which makes the question be true and get likelihood from that
'''


Example

Q - Will the temperature be above 20C on May 24, 2024 in Berlin?

Outcomes:
1 - The temperature will be above 20C on May 24, 2024 in Berlin.
2 - The temperature will be exactly 20C on May 24, 2024 in Berlin.
3 - The temperature will be below 20C but above 10C on May 24, 2024 in Berlin.
4 - The temperature will be below 10C but above 0C on May 24, 2024 in Berlin.
5 - The temperature will be below 0C on May 24, 2024 in Berlin.

We process the LLMs for each outcome, and we get the following

1 - {confidence: 0.5, p_yes: 0.6, p_no: 0.4} -> p_yes selected (p_yes > p_no) -> 0.6 * 0.5 = 0.3 
2 - {confidence: 0.9, p_yes: 0.9, p_no: 0.1} -> 
3 - {confidence: 0.4, p_yes: 0.8, p_no: 0.2}
4 - {confidence: 0.3, p_yes: 0.7, p_no: 0.3}
5 - {confidence: 0.4, p_yes: 0.7, p_no: 0.3}

Note that outcome 1 is always a rephrase of the question. So we are only interUested in that outcome.
The idea is, however, that we can "normalize" that outcome by the other probabilities and confidences.

I have 2 ideas I would like to suggest:
-> Get highest probability (max(p_yes,p_no)), normalize by the other probabilities, return that as probability that
the question is true
-> Use confidence as a "weight" for the probability before normalizing like above, i.e. 

$\sum(w_i * p_i) = 1$

and we return the normalized $(w_i * p_i)$ as the probability.